In [156]:
class BasicTokenizer:
    
    def __init__(self):
        self.vocab = {i:[i] for i in range(256)}
        self.encode_dict = {}
        self.vocab_size = 256

    def _most_frequent(self, byt):
        temp = {}
        most_frequent_pair = None        

        for pair in zip(byt, byt[1:]):
            temp[pair] = temp.get(pair, 0) + 1            
            if most_frequent_pair == None or temp[pair] > temp[most_frequent_pair]:
                most_frequent_pair = pair
        
        

        return most_frequent_pair, temp[most_frequent_pair]
    
    def _replace(self, text, pair, pair_id):
        new_text = []        

        f = False
        for i in range(len(text) - 1):
            if f:
                f = False
                continue

            if text[i] == pair[0] and text[i+1] == pair[1]:
                new_text.append(pair_id)
                f = True
                continue

            new_text.append(text[i])
        
        return new_text

    def train(self, text, threshold):        
        ids = list(text.encode())
        count = 10**8

        while count > threshold:
            pair, count = self._most_frequent(ids)
            # print(count)
            self.vocab[self.vocab_size] = pair
            self.encode_dict[pair] = self.vocab_size
            ids = self._replace(ids, pair, self.vocab_size)
            # print(self.vocab_size)
            # print(ids)
            self.vocab_size += 1


    def find_and_replace(self, text):
        encoded = []

        i=0
        f = False
        while i < len(text)-1:
            pair = text[i], text[i+1]
            # print(pair)
            if pair in self.encode_dict:
                f = True
                encoded.append(self.encode_dict[pair])
                i+=2
                continue
            encoded.append(text[i])
            i+=1
        return f, encoded
    
    
    def encode(self, text, string=False):
        
        if string:
            text = list(text.encode())

        f, encoded = self.find_and_replace(text)

        if not f:
            return encoded
        
        return self.encode(encoded)
    

    def find_and_decode(self, ids):
        decoded = []

        f = False
        for id in ids:
            if id > 255:
                f = True
                # print("hereee")
                # print(self.vocab[id])
                decoded.append(self.vocab[id][0])
                decoded.append(self.vocab[id][1])
                continue
            decoded.append(id)
        return f, decoded

    def decode(self, ids):
        f, decoded = self.find_and_decode(ids)

        if not f:
            return ''.join(bytes(decoded).decode())
            # return decoded
        
        return self.decode(decoded)

In [157]:
tokenizer = BasicTokenizer()
s = "SS stands for Schutzstaffel, which was a Nazi security and military organization. Adolf Hitler established the SS in April 1925 as a personal guard unit. The SS was responsible for many crimes against humanity and war crimes during World War II, including:"
tokenizer.train(s, 1)

In [158]:
len(tokenizer.encode_dict)

33

In [159]:
len(tokenizer.encode(s,True))

170

In [160]:
len(s.encode())

256

In [161]:
tokenizer.encode(s,True)[:10]

[288, 261, 100, 269, 280, 83, 270, 117, 116, 122]

In [162]:
tokenizer.decode(tokenizer.encode(s,True))

'SS stands for Schutzstaffel, which was a Nazi security and military organization. Adolf Hitler established the SS in April 1925 as a personal guard unit. The SS was responsible for many crimes against humanity and war crimes during World War II, inclu'